In [1]:
%load_ext autoreload
%autoreload 2

In [159]:
JSD_Path = f'/Users/xiaoyu/Projects/absbox.work/data/chinabond/docs/22JSD03/吉时代2022年第三期个人汽车抵押贷款资产支持证券受托机构报告第9期.pdf'

In [31]:
from PyPDF2 import PdfReader
import re
  
def read_pdf(p):
    reader = PdfReader(p)
    full_text = "\n".join([p.extract_text() for p in reader.pages])
    return full_text 

In [29]:
print(read_pdf(JSD_Path))

 1 / 11  
吉时代2022年第三期个人汽车抵押贷款  
资产支持证券  
受托机构报告 (第9期) 
(2023年6月16日) 
 
受托机构管理信托财产应恪尽职守，履行诚实、信用、谨慎、
有效管理义务。依据信托合同规定管理信托财产所产生的风
险，由信托财产承担，即由委托人交付的财产以及由受托 机构
对该财产运用后形成的财产承担；受托 机构违背信托合同、处
理信托事务不当使信托财产受到损失，由受托 机构赔偿。 
 
 
受托人：  上海国际信托有限公司  
报告日期：  2023年6月16日 
  
  
受托人地址：  中国上海市 九江路111号 
电话： 021-23131111 
传真： 021-63237929  
公司网址：  http://www.shanghaitrust.com/  
  
 2 / 11 项目名称：吉时代 2022年第三期个人汽车抵押贷款资产支持证券受托机构报
告 
吉时代2022年第三期个人汽车抵押贷款资产支持证券 受托机构报告  
 2022年9月9日，经发起机构吉致汽车 金融有限公司、本公司承诺受
托管理的“吉时代 2022年第三期个人汽车抵押贷款证券化信托”依法
成立。 作为受托机构， 根据 2023年6月9日吉致汽车金融有限公司 《吉
时代2022年第三期个人汽车抵押贷款资产支持证券贷款服务机构报
告》、2023年6月14日中国银行股份有限公司上海市分行《资金保管
报告》，本公司现向您报告本信托信托财产的管理、运用、处分及收益
情况。本公司对本报告全部内容的真实性、准确性、完整性负责。   
 目录 页码 
 一、证券概况  4 
二、资产池(2023年5月1日至2023年5月31日)情况 5 
三、基础资产总体情况  9 
四、信用增级情况  9 
五、报告接收情况说明  10 
六、差错更正说明  10 
七、其它重要事项  10 
注： 1. 本报告内容仅在以下网站披露：  
北京金融资产交易所官方网站 (www.cfae.cn)  
中国货币网 (www.chinamoney.com.cn)  
中国债券信息网 (www.chinabond.com.cn)   
2.本报告内容根据贷款月度服务机构报告等报告内容编制。  
3.本报告金额单位均以人民币元计，本报告期限单位均以月计。  
4.收款期间为 ：20

In [30]:
t = read_pdf(JSD_Path)

In [106]:
def read_rpt_date(x):
    x = re.search(r"报告日期：\s+(\S+)",t).groups()
    if x:
        return x[0]
    
def read_period_dates(x):
    start_date = re.search(r"本期期初日\s+(\S+)",t).groups()
    end_date = re.search(r"本期期末日\s+(\S+)",t).groups()
    return start_date[0],end_date[0]

def read_bond_payments(x):
    r = re.search(r"(证券名称  信托设立日余额.*?)\n\s?\n",t,re.DOTALL).groups()[0].split("\n")
    r = [ re.split("\s+",_) for _ in r[1:] ]
    curr_bal = {_[0]:_[4].replace(",","")  for _ in r }
    return curr_bal

def read_bond_int_payments(x):
    r = re.search(r"证券名称  证券执行.*总支付金额\s+(.*?)\s+合计",t,re.DOTALL).groups()[0].split("\n")
    r = {y[0]:y[1].replace("%","") for y in [_.split(" ") for _ in r]}
    return r

In [191]:
import datetime

In [197]:
y,m,d = re.split(r"\D",read_rpt_date(t))[:-1]
datetime.date(int(y),int(m),int(d)).strf("%Y-%M-%d")

AttributeError: 'datetime.date' object has no attribute 'strf'

In [193]:
def convert_cf_table_flows(x):
    def to_date(y):
        match y:
            case _ if re.match(r"\d{4}年\d{2}月\d{2}日",y):
                (yyyy,mm,dd) = re.match(r"(\d{4})年(\d{2})月(\d{2})日",'2023年06月30日').groups()
                return f"{yyyy}-{mm}-{dd}"
            case _:
                raise RuntimeError(f"Failed to match {y}")
    d,b_bal,principal,interest,e_bal = x
    return [to_date(d),float(e_bal),float(principal),float(interest)]

In [177]:
def read_cf_table(x):
    # 信托核算日 期初本金总余额 本期应收本金 本期应收利息 期末本金总余额 
    r = re.search("现金流归集表.*?信托核算日\s.*?\n(.*?)\d\s+\/\s+\d{1,2}",t,re.DOTALL).groups()[0].split("\n")
    r = [ _.replace(",","") for _ in r]
    r = [ _.rstrip(" ") for _ in r]
    r = [ re.split(r"\s+",_) for _ in r]
    r = [ convert_cf_table_flows(_) for _ in r if len(_)==5]
    return r